<a href="https://colab.research.google.com/github/annndzrh/hangman/blob/master/NLP_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np


file_path = "en_ewt-ud-train.conllu"

# Parse the dataset
sentences = []
pos_tags = []
sentence = []
tag = []

with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

for line in lines:
    if line.startswith("#"):
        continue
    if line.strip() == "":
        if sentence:
            sentences.append(sentence)
            pos_tags.append(tag)
            sentence = []
            tag = []
    else:
        parts = line.split("\t")
        if len(parts) > 3:
            sentence.append(parts[1])
            tag.append(parts[3])

if sentence:
    sentences.append(sentence)
    pos_tags.append(tag)

# Create vocabularies for words and tags
vocab_size = 10000
embedding_dim = 16
max_length = 20
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

word_tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
word_tokenizer.fit_on_texts(sentences)

tag_tokenizer = Tokenizer(lower=False)
tag_tokenizer.fit_on_texts(pos_tags)

# Convert words and tags to sequences
word_index = word_tokenizer.word_index
tag_index = tag_tokenizer.word_index

X = word_tokenizer.texts_to_sequences(sentences)
y = tag_tokenizer.texts_to_sequences(pos_tags)

# Pad sequences
X = pad_sequences(X, maxlen=max_length, padding=padding_type, truncating=trunc_type)
y = pad_sequences(y, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Convert tags to categorical
num_tags = len(tag_index) + 1
y = [to_categorical(i, num_classes=num_tags) for i in y]
y = np.array(y)

# Split the data into training and testing sets
training_size = int(len(X) * 0.8)
X_train, X_test = X[:training_size], X[training_size:]
y_train, y_test = y[:training_size], y[training_size:]

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(num_tags, activation='softmax'))
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")



Epoch 1/10
283/283 [==============================] - 23s 46ms/step - loss: 1.9842 - accuracy: 0.4139 - val_loss: 0.8958 - val_accuracy: 0.7509
Epoch 2/10
283/283 [==============================] - 9s 31ms/step - loss: 0.5891 - accuracy: 0.8370 - val_loss: 0.3937 - val_accuracy: 0.8909
Epoch 3/10
283/283 [==============================] - 10s 36ms/step - loss: 0.2932 - accuracy: 0.9213 - val_loss: 0.3167 - val_accuracy: 0.9061
Epoch 4/10
283/283 [==============================] - 10s 36ms/step - loss: 0.2089 - accuracy: 0.9405 - val_loss: 0.2924 - val_accuracy: 0.9127
Epoch 5/10
283/283 [==============================] - 10s 36ms/step - loss: 0.1686 - accuracy: 0.9503 - val_loss: 0.2827 - val_accuracy: 0.9177
Epoch 6/10
283/283 [==============================] - 12s 42ms/step - loss: 0.1428 - accuracy: 0.9577 - val_loss: 0.2828 - val_accuracy: 0.9179
Epoch 7/10
283/283 [==============================] - 9s 33ms/step - loss: 0.1254 - accuracy: 0.9625 - val_loss: 0.2839 - val_accuracy: 0

In [6]:
# Predict POS tags for the input sentence
print("Enter text.")
text = input()
sentences = [text.split()]
sequences = word_tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


predictions = model.predict(padded)
predicted_tags = np.argmax(predictions, axis=-1)

# Convert indices back to words and tags
words = [word for word in text.split()]
tags = [tag_tokenizer.index_word[idx] for idx in predicted_tags[0] if idx != 0]

print("Words:", words)
print("Predicted POS tags:", tags)

Enter text.
Artificial Intelligence (AI) is an evolving technology that tries to simulate human intelligence using machines.
1/1 [==============================] - 0s 24ms/step
Words: ['Artificial', 'Intelligence', '(AI)', 'is', 'an', 'evolving', 'technology', 'that', 'tries', 'to', 'simulate', 'human', 'intelligence', 'using', 'machines.']
Predicted POS tags: ['ADJ', 'NOUN', 'NOUN', 'AUX', 'DET', 'NOUN', 'NOUN', 'PRON', 'VERB', 'PART', 'VERB', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'PROPN', 'PROPN', 'PROPN', 'PROPN', 'PROPN']
